In [9]:
from typing import Tuple
from abc import ABC, abstractmethod
from plyfile import PlyData
import numpy as np
import open3d as o3d

class PointCloudLoader(ABC):
    def __call__(self, pc_path: str) -> Tuple[np.array, np.array]:
        pass

class ColmapLoader(PointCloudLoader):
    def __call__(self, pc_path: str):
        pcd = o3d.io.read_point_cloud(pc_path)
        points = np.asarray(pcd.points)
        colors = np.asarray(pcd.colors)

        return points, colors

class GaussianLoader(PointCloudLoader):
    def __call__(self, pc_path: str):
        pcd = PlyData.read(pc_path)

        x = pcd.elements[0]['x']
        y = pcd.elements[0]['y']
        z = pcd.elements[0]['z']

        points = np.stack([x, y, z], axis=1)
        
        raw_r = pcd.elements[0]['f_dc_0']
        raw_g = pcd.elements[0]['f_dc_1']
        raw_b = pcd.elements[0]['f_dc_2']

        r = self.read_color_from_harmonics(raw_r)
        g = self.read_color_from_harmonics(raw_g)
        b = self.read_color_from_harmonics(raw_b)
        
        colors = np.stack([r, g, b], axis=1)

        return points, colors

    def read_color_from_harmonics(self, color_harmonics):
        C0 = 0.28209479177387814
        return np.clip(0.5 + (C0 * color_harmonics), a_min=0, a_max=1)
        

In [10]:
import json

class Camera:
    AXES_SIZE = 0.2

    def __init__(self, camera_data):
        self.position = np.array(camera_data["position"])
        
        rotation = np.array(camera_data["rotation"])
        self.rotation_x = rotation[:, 0]
        self.rotation_y = rotation[:, 1]
        self.rotation_z = rotation[:, 2]

class CameraLoader:
    def __call__(self, camera_path: str):
        with open(camera_path) as f:
            camera_json = json.loads(f.read())
        return [
            Camera(camera_data)
            for camera_data in camera_json
        ]

In [11]:
from enum import Enum
from __future__ import annotations

class CalibrationPoint(Enum):
    BLUE = 1
    ORANGE = 2
    GREEN = 3

class ColorAligner:
    CALIBRATION_POINTS = {
        CalibrationPoint.BLUE: np.array([99, 176, 204]) / 256,
        CalibrationPoint.ORANGE: np.array([232, 197, 99]) / 256,
        CalibrationPoint.GREEN: np.array([132, 206, 182]) / 256
    }

    def __call__(self, color: Color):
        for calibration_point in self.CALIBRATION_POINTS.keys():
            if self.is_close_to(color, calibration_point):
                return calibration_point
        return None

    def is_close_to(self, color: Color, calibration_point: CalibrationPoint):
        return self.get_dist(color, self.CALIBRATION_POINTS[calibration_point]) < 0.1

    def get_dist(self, color1: np.array, color2: np.array):
        return abs(color1[0] - color2[0]) + abs(color1[1] - color2[1]) + abs(color1[2] - color2[2])

In [12]:
from sklearn.cluster import DBSCAN
import itertools

class Transformer:
    def __call__(self, point: np.array):
        pass

class NoTransformer(Transformer):
    def __call__(self, point: np.array):
        return point

    def get_rotation_matrix(self, points: np.array, colors: np.array):
        return None

class CalibrationTransformer(Transformer):
    def __init__(self):
        self.color_aligner = ColorAligner()

    def get_rotation_matrix(self, points: np.array, colors: np.array):
        
        self.colored_points = {
            CalibrationPoint.BLUE: [],
            CalibrationPoint.ORANGE: [],
            CalibrationPoint.GREEN: []
        }

        for i, color in enumerate(colors):
            centroid = self.color_aligner(color)
            if centroid is not None:
                self.colored_points[centroid].append(points[i])

        self.centroids = self.find_centroids()

        self.unit_dist = np.linalg.norm(self.centroids[CalibrationPoint.BLUE] - self.centroids[CalibrationPoint.ORANGE]) / 2
        x_vec = (self.centroids[CalibrationPoint.BLUE] - self.centroids[CalibrationPoint.ORANGE]) / (self.unit_dist*2)
        y_vec = (self.centroids[CalibrationPoint.GREEN] - self.centroids[CalibrationPoint.ORANGE]) / (self.unit_dist*2)
        z_vec = np.cross(x_vec, y_vec)

        self.rotation_matrix = np.linalg.inv(np.column_stack((x_vec, y_vec, z_vec)))

    def find_centroids(self):
        blue_candidate_cluster_centroids = self.get_candidate_cluster_centroids(np.array(self.colored_points[CalibrationPoint.BLUE]))
        orange_candidate_cluster_centroids = self.get_candidate_cluster_centroids(np.array(self.colored_points[CalibrationPoint.ORANGE]))
        green_candidate_cluster_centroids = self.get_candidate_cluster_centroids(np.array(self.colored_points[CalibrationPoint.GREEN]))

        # find a combination of cluster centroids that give the highest orthogonality and length similarity
        best_score = float('inf')
        best_combination = None
        for blue_centroid, orange_centroid, green_centroid in itertools.product(
            blue_candidate_cluster_centroids,
            orange_candidate_cluster_centroids,
            green_candidate_cluster_centroids
        ):
            score = self.evaluate_cluster_combination(blue_centroid, orange_centroid, green_centroid)
            if score < best_score:
                best_score = score
                best_combination = (blue_centroid, orange_centroid, green_centroid)

        return {
            CalibrationPoint.BLUE: best_combination[0],
            CalibrationPoint.ORANGE: best_combination[1],
            CalibrationPoint.GREEN: best_combination[2]
        }
    
    def get_candidate_cluster_centroids(self, points):
        # Step 1: Apply DBSCAN
        dbscan = DBSCAN(eps=0.5, min_samples=min(len(points), 5))  # Adjust eps and min_samples as needed
        labels = dbscan.fit_predict(points)

        # Step 2: Mask out noise points (where label is -1)
        valid_labels = labels != -1
        valid_points = points[valid_labels]
        valid_labels = labels[valid_labels]

        # Step 3: Calculate centroids using numpy
        unique_labels = np.unique(valid_labels)
        centroids = np.array([valid_points[valid_labels == label].mean(axis=0) for label in unique_labels])

        return centroids

    def evaluate_cluster_combination(self, blue_centroid: np.array, orange_centroid: np.array, green_centroid: np.array):
        x_axis = blue_centroid - orange_centroid
        y_axis = green_centroid - orange_centroid

        dot = abs(np.dot(x_axis, y_axis))
        x_len = np.linalg.norm(x_axis)
        y_len = np.linalg.norm(y_axis)
        len_diff = max(abs(x_len - y_len)/y_len, abs(y_len - x_len) / x_len)

        return dot# + 10*len_diff

    def __call__(self, point):
        return np.dot(self.rotation_matrix, point - self.centroids[CalibrationPoint.ORANGE]) - np.array([self.unit_dist, self.unit_dist, 0])

In [13]:
class Filterer:
    def __call__(self, point: np.array, color: np.array):
        pass

class CalibrationPointsFilterer(Filterer):
    def __init__(self):
        self.color_aligner = ColorAligner()

    def __call__(self, point: np.array, color: np.array):
        return self.color_aligner(color) is not None

class NoFilterer:
    def __call__(self, point: np.array, color: np.array):
        return True

class CalibrationAreaFilterer(Filterer):
    def __call__(self, point: np.array, color: np.array):
        return abs(point[0]) < 1 and abs(point[1]) < 1 and point[2] < -0.01

In [14]:
class CameraLineSetBuilder:
    def __init__(self, transformer: Transformer):
        self.transformer = transformer
    
    def __call__(self, cameras: List[Camera], axes_size=0.2):
        points = []
        lines = []
        colors = []
        
        for i, camera in enumerate(cameras):
            points.append(self.transformer(camera.position))

            end_x = camera.position + axes_size*camera.rotation_x
            end_y = camera.position + axes_size*camera.rotation_y
            end_z = camera.position + axes_size*camera.rotation_z

            points.append(self.transformer(end_x))
            points.append(self.transformer(end_y))
            points.append(self.transformer(end_z))
            
            lines.extend([[4*i, 4*i+1], [4*i, 4*i+2], [4*i, 4*i+3]])
            colors.extend([
                [1, 0, 0], [0, 1, 0], [0, 0, 1]
            ])
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector(points)
        line_set.lines = o3d.utility.Vector2iVector(lines)
        line_set.colors = o3d.utility.Vector3dVector(colors)

        return line_set

class PointCloudBuilder:
    def __init__(self, transformer: Transformer, filterer: Filterer):
        self.transformer = transformer
        self.filterer = filterer
    
    def __call__(self, points: np.array, colors: np.array):
        transformed_points = []
        transformed_colors = []
        for point, color in zip(points, colors):
            transformed_point = self.transformer(point)
            if self.filterer(transformed_point, color):
                transformed_points.append(transformed_point)
                transformed_colors.append(color)
        
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(transformed_points)
        pcd.colors = o3d.utility.Vector3dVector(transformed_colors)

        return pcd

In [15]:
from typing import List

class Scene:
    def load_point_cloud(self, point_cloud_loader: PointCloudLoader, pc_path: str):
        self.points, self.colors = point_cloud_loader(pc_path)

    def load_camera_cloud(self, camera_loader: CameraLoader, camera_path: str):
        self.cameras = camera_loader(camera_path)

    def set_transformer(self, transformer: Transformer):
        transformer.get_rotation_matrix(self.points, self.colors)
        self.transformer = transformer

    def set_filterer(self, filterer: Filterer):
        self.filterer = filterer

    def visualise(self):
        vis = o3d.visualization.Visualizer()
        vis.create_window(window_name="3D Point Cloud Visualization")

        axis = o3d.geometry.TriangleMesh.create_coordinate_frame(
            size=1.0,  # Axis length (you can adjust the size)
            origin=[0, 0, 0]  # Set origin to [0, 0, 0]
        )

        vis.add_geometry(PointCloudBuilder(self.transformer, self.filterer)(self.points, self.colors))
        vis.add_geometry(axis)
        vis.add_geometry(CameraLineSetBuilder(self.transformer)(self.cameras))

        view_control = vis.get_view_control()

        target_point = np.array([0, 0, 0])
        view_control.set_lookat(target_point)

        vis.run()

In [16]:
scene = Scene()
# scene.load_point_cloud(GaussianLoader(), "../colmap-outputs/model-own-6/point_cloud/iteration_30000/point_cloud_backup.ply")
scene.load_point_cloud(ColmapLoader(), "../colmap-outputs/model-own-6/input.ply")
scene.load_camera_cloud(CameraLoader(), "../colmap-outputs/model-own-6/cameras.json")
scene.set_transformer(CalibrationTransformer())
scene.set_filterer(CalibrationAreaFilterer())
# scene.visualise()

In [17]:
scene.visualise()

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


2024-11-19 21:15:24.089 Python[15051:2901153] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/d3/24c_clj17296vpj3rnm0k8vc0000gn/T/org.python.python.savedState
